In [ ]:
import ogr
import os

### Setup a shapefile for our use

In [ ]:
shapefile = os.path.join('NHD_H_Idaho_State_Shape','Shape','WBDHU8.shp')

### Open the shapefile using ogr, the vector component of gdal

A simple demonstration of [opening a shapefile using GDAL/OGR](https://gdal.org/python/osgeo.ogr-module.html#Open)*
*The official API docs are difficult to link to, and not much help directly. Better to use the [docs viewer online](https://gdal.org/python/index.html)

In [ ]:
shp_source = ogr.Open(shapefile)
shp_layer = shp_source.GetLayer()

### Get all the attributes in the shapefile

In [ ]:
layerDefinition = shp_layer.GetLayerDefn()
for i in range(layerDefinition.GetFieldCount()):
    print(layerDefinition.GetFieldDefn(i).GetName())

### Print data about a specific attribute

Filter the data and print values of those objects which passed

In [ ]:
shp_layer.SetAttributeFilter ( "AreaAcres > 900000" )
for feat in shp_layer:
    print(f'{feat.GetFieldAsString("HUC8")} has {feat.GetFieldAsInteger("AreaAcres")} acres of area')

### Additional Material

[This repository](https://ceholden.github.io/open-geo-tutorial/python/chapter_4_vector.html) has a really good overview of working with shape data in using ogr. See cell 2 specifically

## Simplify Things With Geopandas

Ok, so we can read the data in the shapefile using gdal's ogr driver. It works, but it's not really elegant. Enter GeoPandas. The awesome processing power of pandas made ready to work with geospatial data.

### Load the data to a geodataframe and explore the data

use [read_file](https://geopandas.org/reference/geopandas.read_file.html) to get the shapefile into a [GeoDataFrame](https://geopandas.org/data_structures.html#geodataframe)

In [ ]:
import geopandas as gpd
gdf = gpd.read_file(shapefile)
print(gdf)

In [ ]:
gdf.head()

### Searching and filtering is as simple as using Pandas functionality

In [ ]:
gdf.loc[gdf['AreaAcres'] > 90000]

### Numbers are great and all...but I wanna SEE the data

After all, it is a map we're playing with. Luckily, [GeoPandas makes it easy](https://geopandas.org/mapping.html) using the [plot](https://geopandas.org/reference.html#geopandas.GeoSeries.plot) command. Like many other great Python libraries, it makes use of matplotlib under the hood.

In [ ]:
fig_size = (10,12)
gdf.plot(figsize=fig_size)

### Let's pick a column to display

Drill down into the data a little and see what it looks like

In [ ]:
gdf.plot(column='AreaAcres', figsize=fig_size)

### Let's add a legend

It's always nice to know what those pretty colors mean. Adding a legend is as easy as setting the argument to True

In [ ]:
gdf.plot(column='AreaAcres', legend=True, figsize=fig_size)

### Uh Oh! The legend looks a little...out of proportion

Let's fix the legend with a little help from our friend, the locatable axis

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(1, 1, figsize=fig_size)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
gdf.plot(column='AreaAcres', ax=ax, legend=True, cax=cax)

### Let's look at another shapefile

Here we have the data for bodies of water

In [ ]:
shapefile2 = os.path.join('NHD_H_Idaho_State_Shape','Shape','NHDWaterbody.shp')
gdf2 = gpd.read_file(shapefile2)
print(gdf2)

In [ ]:
gdf2.head()

### Let's use Panda's sorting and filtering to view a portion of the data

In [ ]:
gdf2.loc[gdf2['AreaSqKm'] > 90].sort_values('GNIS_Name')

### Now we can plot just our filtered data

In [ ]:
gdf2.loc[gdf2['AreaSqKm'] > 90].sort_values('GNIS_Name').plot(figsize=fig_size)

### Play around with the data

See if you can identify the largest body of water in the Waterbody shapefile.

In [ ]:
# first sort the dataframe by AreaSqKm, descending.
# then get a dataframe of only the first row and plot it
largest = gdf2.sort_values('AreaSqKm', ascending=False).iloc[[0]]
largest.plot()

### Combine Shapefiles into a single plot

With a few additions and adjustments to our previous plot code, we can add multiple shape files into one plot

In [ ]:
f, ax = plt.subplots(1, 1, figsize=fig_size)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
gdf.plot(ax=ax)
gdf2.plot(ax=ax, column='OBJECTID', cax=cax, legend=True)
plt.show()